In [85]:
import os, pprint
from ordered_set import OrderedSet

template_api_path = "../sample_files/base-template/"
source_api_path = "../sample_files/api-current/"
destination_api_path = "../sample_files/api-destination/"

filename = "build.gradle"
template_file = os.path.join(template_api_path, filename)
source_file = os.path.join(source_api_path, filename)
destination_file = os.path.join(destination_api_path, filename)

In [86]:
# with open(template_file) as file:
#     for line in file:
#         print(line.rstrip())

In [87]:
with open(template_file, "r") as f, open(source_file, "r+") as out:
    template_line_start = 0
    template_line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            template_line_start = index
        if template_line_start !=0 and line.startswith("}"):
            template_line_end = index
            break

    source_line_start = 0
    source_line_end = 0
    for index_out, line_out in enumerate(out):
        if line_out.startswith("dependencies {"):
            source_line_start = index_out
        if source_line_start !=0 and line_out.startswith("}"):
            source_line_end = index_out
            break
            

In [88]:
with open(template_file) as f:
    i = 0
    template_dict_list = []
    template_list = []
    for index, line in enumerate(f):
        if index > template_line_start and index < template_line_end and line !='\n':
            template_list.append(line)
            data = {}
            template_split_line = line.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
#             print("Line {}: {}".format(index, template_split_line))
            i += 1
            if i == 1 or i == 8 or i >= 13:
                line_data = template_split_line.split(":")
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    data["prefix"] = line_data[0]
                    data["artifact"] = line_data[len(line_data)-2]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 1
            elif i == 10:
                line_data = template_split_line.replace(",", ":").split(":")
                if len(line_data) > 1:
                    ver_list = line_data[len(line_data)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    data["prefix"] = line_data[0]
                    data["artifact"] = line_data[len(line_data)-3]
                    data["version"] = line_data[len(line_data)-1]
                    data["version_sum"] = sum(ver_list)
                    data['template_line'] = line
                    data['rule'] = 2
            else:
                line_data = template_split_line.split(":")
                if len(line_data) > 1:
                    data["prefix"] = line_data[0]
                    data["artifact"] = line_data[len(line_data)-1]
                    data["version"] = None
                    data["version_sum"] = 0
                    data['template_line'] = line
                    data['rule'] = None
            template_dict_list.append(data)
                
# pprint.pprint(template_dict_list)
# pprint.pprint(template_list)

In [89]:
with open(source_file, "r") as f:
    template_set = OrderedSet()
    unchanged_set = OrderedSet()
    changed_set = OrderedSet()
    for index_out, line_out in enumerate(f):
        if index_out > source_line_start and index_out < source_line_end:
#             print("Line {}: {}".format(index_out, line_out.strip()))
            for l in template_dict_list:
                if l['artifact'] in line_out:
                    template_set.add(l['template_line'])
                    changed_set.add(line_out)
                else:
                    unchanged_set.add(line_out)
                    
# pprint.pprint(unchanged_set)
# pprint.pprint(changed_set)
diff = (unchanged_set - changed_set)
pprint.pprint(diff)

OrderedSet(['\n', "\tcompile 'org.springframework.boot:spring-boot-starter-data-jpa'\n", '\t// https://mvnrepository.com/artifact/com.oracle.database.jdbc/ojdbc8\n', "\tcompile 'com.oracle.database.jdbc:ojdbc8'\n", '\t// https://mvnrepository.com/artifact/io.fabric8/openshift-client\n', "\timplementation group: 'io.fabric8', name: 'openshift-client', version: '5.12.2'\n"])


In [90]:
compose_template = []
compose_template.append(['\n\ndependencies {\n'])
compose_template.append(template_set)
compose_template.append(diff)

compose_template = [line.replace('compile', 'implementation') for level in compose_template for line in level]
# pprint.pprint(compose_template)
    
with open(source_file, "r") as f:
    final_data_list = []
    for index_, line_ in enumerate(f):
        if index_ > line_start_out-1 and index_ < line_end_out+1:
            continue
        final_data_list.append(line_)
# print(final_data_list)

for line in compose_template:
    final_data_list.append(line)
    
pprint.pprint(final_data_list)

["apply plugin: 'java'\n",
 "apply plugin: 'maven'\n",
 "apply plugin: 'eclipse'\n",
 "apply plugin: 'com.github.sherter.google-java-format'\n",
 "apply plugin: 'org.springframework.boot'\n",
 "apply plugin: 'io.spring.dependency-management'\n",
 'apply from: "${rootDir}/gradle/common/jacoco.gradle"\n',
 'apply from: "${rootDir}/gradle/common/repositories.gradle"\n',
 '\n',
 "group = 'com.capgemini.api'\n",
 "version = '0.0.1-SNAPSHOT'\n",
 "sourceCompatibility = '1.8'\n",
 '\n',
 'buildscript {\n',
 '\trepositories {\n',
 '\t\tmaven {\n',
 "\t\t\turl 'https://repo.spring.io/milestone'\n",
 '\t\t}\n',
 '\t}\n',
 '\tdependencies {\n',
 '\t\tclasspath '
 "'io.spring.gradle:dependency-management-plugin:1.0.2.RELEASW'\n",
 '\t\tclasspath '
 "'org.springframework.boot:spring-boot-gradle-plugin:1.0.10.RELEASE'\n",
 '\t\tclasspath '
 "'com.github.sherter.googlejavaformatgradleplugin:google-java-format-gradle-plugin:0.9-SNAPSHOT'\n",
 '\t}\n',
 '}\n',
 '\n',
 '\n',
 'ext {\n',
 '\tset(\'spring

In [92]:
with open(destination_file, "w+") as f:
    for line in final_data_list:
        f.writelines(line)